<a href="https://colab.research.google.com/github/moonsejin315/Python/blob/main/LangChain/LangChain_04_%EB%9E%AD%EC%B2%B4%EC%9D%B8_%EA%B8%B0%EB%B0%98_%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -upgrade -quiet langchain-core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain_openai

In [26]:
#튜플이나 리스트같은 '컬렉션타입' 데이터에서 원하는 인덱스의 요소를 추출하는 데 사용
from operator import itemgetter

#러너블 클래 : 함수를 감싸서 체인화 할 수 있게 해줌
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

#Output(결과물)을 constent만 가져올 수 있도록 변환
from langchain_core.output_parsers import StrOutputParser

#특정한 대화 양식(템플릿)을 정의하여 쉽게 질의응답 할 수 있도록 해줌
#우리가 나눈 대화를 관리하는 데 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#openai와의 대화형 질의 응답
from langchain_openai import ChatOpenAI

#openai와 나눈 대화를 저장하고 관리하는데 사용하는 클래스
#"이전 대화를 기반으로" 현재의 대화를 이끌어나가도록 함
from langchain.memory import ConversationBufferWindowMemory

In [27]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## 챗봇을 정의

In [46]:
system_prompt = """
  너는 나랑 친구인 상담사야.
  내가 물어보는 질문에 대해서 친절하게 한국어로 답변해줘.
"""

In [47]:
#리스트로 감싸서 튜플로 전달한다.
chat_prompt_template = ChatPromptTemplate.from_messages([
  ("system", system_prompt),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{user_input}")
])

#ConversationBufferWindowMemory는 몇 개(k)까지 기억할지 정해줄 수 있음
#총 3개의 이전 대화 쌍을 기억함.
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [48]:
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

In [53]:
chain = ({"user_input":RunnablePassthrough()} |
        RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
        | chat_prompt_template
        | chat_model
        | output_parser)

In [54]:
def load_memory(_):
  return memory.load_memory_variables(_)['history']

In [55]:
def chat_with_user(user_message):
  ai_message = chain.invoke(user_message)

  memory.save_context({"input":user_message}, {"output":ai_message})
  print(memory.load_memory_variables({}))

  return ai_message

In [56]:
chat_with_user("안녕. 오늘 비 오는데 우산 챙겼니?")

{'history': [HumanMessage(content='안녕. 오늘 비 오는데 우산 챙겼니?'), AIMessage(content='안녕. 비 오는 날 우산을 챙겼다면 좋겠네요. 비 오는 날 우산을 챙기지 않았다면 지금이라도 챙기는 것이 좋을 것 같아요. 혹시 우산을 챙겼는지 확인해보세요!')]}


'안녕. 비 오는 날 우산을 챙겼다면 좋겠네요. 비 오는 날 우산을 챙기지 않았다면 지금이라도 챙기는 것이 좋을 것 같아요. 혹시 우산을 챙겼는지 확인해보세요!'

In [58]:
while True:
  user_message = input("USER >>")
  if user_message == 'quit':
    break
  ai_message = chat_with_user(user_message)
  print(f"AI >>{ai_message}")

USER >>여행지 추천해줘
{'history': [HumanMessage(content='안녕. 오늘 비 오는데 우산 챙겼니?'), AIMessage(content='안녕. 비 오는 날 우산을 챙겼다면 좋겠네요. 비 오는 날 우산을 챙기지 않았다면 지금이라도 챙기는 것이 좋을 것 같아요. 혹시 우산을 챙겼는지 확인해보세요!'), HumanMessage(content='심심한데 오늘 뭐 하면 좋을까?'), AIMessage(content='심심할 때 하는 것은 개인마다 다를 수 있지만, 몇 가지 추천해 드릴게요. \n1. 취미 활동: 취미를 즐기는 것도 좋은 방법이에요. 음악 듣기, 그림 그리기, 책 읽기 등 자신의 취미에 집중해보세요.\n2. 운동: 신체 활동은 마음의 휴식이 될 수 있어요. 집에서 요가나 스트레칭을 해보는 것도 좋은 방법이에요.\n3. 요리: 새로운 요리를 도전해보는 것도 재미있을 수 있어요. 집에 있는 재료로 요리를 해보는 것은 즐거운 경험이 될 거예요.\n이런 것들을 시도해보면서 오늘을 유익하고 즐겁게 보낼 수 있을 거에요.'), HumanMessage(content='여행지 추천해줘'), AIMessage(content='여행지를 추천해드릴 때 여러 가지 요소를 고려해야 하지만, 몇 가지 인기 있는 여행지를 추천해드릴게요.\n1. 제주도: 한국에서 가장 인기 있는 여행지 중 하나로 자연 풍경과 맛있는 음식을 즐길 수 있어요. 한라산 등반이나 해변 산책을 즐길 수 있어요.\n2. 부산: 바다와 산이 어우러진 부산은 맛있는 해산물과 풍부한 문화를 경험할 수 있는 도시에요. 해운대 해변이나 부산 타워를 방문해보세요.\n3. 경주: 역사와 문화가 곁들어진 경주는 한국의 역사적인 도시로 유네스코 세계문화유산으로 등재되어 있어요. 석굴암이나 불국사를 방문해보세요.\n이런 곳들을 방문하면 한국의 다양한 매력을 경험할 수 있을 거에요.')]}
AI >>여행지를 추천해드릴 때 여러 가지 요소를 고려해야 하지만, 몇 가지 인기 있는 여행지를 추천해드릴게

In [ ]:
!pip install gradio

In [60]:
import gradio as gr

import random
import time

def respond(user_message, chat_view):
  ai_message = chat_with_user(user_message)
  chat_view.append((user_message, ai_message))

  time.sleep(1)
  return "", chat_view

with gr.Blocks() as demo:
  chat_view = gr.Chatbot(label = "채팅창")

  user_textbox = gr.Textbox(label = "입력")

  user_textbox.submit(respond, [user_textbox, chat_view], [user_textbox, chat_view])
demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d277f1a69273a977c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
